In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.upgrade_sdc import UpgradeSDC
from model.sdc import SDC

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TrueCoffee'

corpus, labels = load_corpus('../data/' + file_name + '.txt')

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/tokenized/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

Total documents 350
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.extend(['นี้'])
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 948 words
filter frequent words: 370 words
filter letter words: 368 words
filter stop words: 219 words


In [4]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

#### Doc2Vec

In [6]:
tagged_corpus = [TaggedDocument(doc, [i]) for i, doc in enumerate(idx_corpus)]
model = Doc2Vec(tagged_corpus, vector_size=average_doc_size, window=4, min_count=2, epochs=100)
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

paragraph_vectors = [model.infer_vector(doc) for doc in idx_corpus]
paragraph_vectors = pandas.DataFrame(paragraph_vectors, dtype=float)

### Clustering

In [7]:
def get_onehot(corpus, weight):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if weight == 'normal':
        weight_corpus = bow_corpus
    elif weight == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        weight_corpus = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(weight_corpus):
        for id, score in doc:
            array[i, id] = score

        if weight == 'normal' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [8]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [9]:
def eval_cluster(onehot_corpus, result):
    label_count = numpy.unique(result['predicted_label'])
    num_cluster = label_count[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [10]:
min_samples = 7
eps = 0.32

onehot_corpus = get_onehot(idx_corpus, 'normal')
# onehot_corpus = get_onehot(filtered_corpus, 'tfidf')

In [11]:
max_compactness = 0
epoch = 15
for i in range(epoch):
    model = UpgradeSDC()
    _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps)

#     model = SDC()
#     _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps)
    
#     marks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=7).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, marks)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)

137.566296070018


In [31]:
centroids = None 
for i in range(1):
    model = UpgradeSDC()
    if centroids:
        centroids = centroids[1:]
    predicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, seeds=centroids)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(onehot_corpus, result)
            
    print(compactness, numpy.unique(result['predicted_label'], return_counts=True)[1][0])

165.43197277018987 73


In [23]:
label_count = numpy.unique(result['predicted_label'], return_counts=True)
num_cluster = label_count[0][-1] + 1
print(label_count, '\n')

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]), array([82, 46, 37, 44, 18, 21,  8, 15, 12, 21, 11, 14, 10, 11])) 



In [24]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len_corpus)
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


### Result

In [25]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [26]:
result.to_csv('../data/results/em/' + file_name + '.csv')

# result = pandas.read_csv('../data/results/2/' + file_name + '.csv')

count = 0
for index, value in result.iterrows():
    if value['marks'] == -1:
        count += 1
print(count)

83


In [27]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), value=0)

1 เรางง สั่งคาปูชิโนหวานน้อย ทำไม่ได้ค่ะ ถ้าลูกค้าอยากได้หวานน้อยต้องสั่งลาเต้ กูงงงงงงง
4 ซื้อลิขสิทธิ์ UCL มาด้วยนะ จ่ายรายเดือนไป แต่ไม่มีให้ดู
12 ผมว่า ผมชอบรสชาติของกาแฟคุณ หลายๆแก้วประทับใจกว่า แบรนด์นางเงือกแต่ว่า แพงไปหว่ะ ไม่เหมาะกะคนไทย (ถึงแม้จะใช้ส่วนลด ลูกค้าทรู แล้วก้ตาม)สังเกตได้ว่า น้อยสาขา ที่คนจะแน่น หรือต้องต่อคิวสัีงกาแฟ เหมือนแบรนด์นางเงือก…See more
17 Wi-Fi ทรูที่บ้าน ใช้ได้แต่กลางคืน กลางวันใช้ไม่ได้ แต่จ่ายพันกว่าบาททุกเดือน ควรปรับปรุง ฝากทรูคอฟฟี่ไปบอกด้วย
21 ทำไมเอาเมนูทวิซที่ปั่นใส่ขนมช็อกโกแลตทวิซคาลาเมลออก 💥🔥🔥🔥โกรธหนักค่ะ เพราะชอบมากกกกกกกก
24 ขนาดตอนเป็นลูกค้าทรูมูฟ ยังไม่กิน ตอนนี้ย้ายค่ายแล้ว อย่าหวังจะได้แดก
28 ขอทำขนมไปส่งได้ไหมคะ นักเรียนบ้านยากจนจะสอนนักเรียนแต่ไม่มีที่ให้โอกาสนักเรียนคะ แต่รับรองขนมอร่อยคะ
42 ตัวอักษรใหญ่ๆ มองเมนูไม่ชัด ไม่กล้าสั่ง กลัวสั่งผิดละเขินน 555+
45 บอกพนักงงานชงกาแฟทำหน้าตาดีๆ เลิกกระแทกแก้วสักที เวลาส่งคืนลูกค้าเยอะๆอ่ะ ละก็ช่วยทำคอมเม้นลูกค้าให้มันเป็นตามที่ขอด้วยอิเวน
48 ราคาที่ตั้งนี่ เพื่ิอ ขายนิล อาร์มสตรองที่ดวงจัน

In [28]:
w2 = Widget(result, 'comment')

ToggleButtons(index=8, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), value=8)

*18 เครื่องดื่มโบราณ อัดน้ำแข็งแน่นมาก ดูด 3-4 ทีหมด น้ำแข็งยังแน่นแก้วอยู่เลย
*29 ซื้อเพราะหาที่นั่งล้วนๆ รสชาติไม่ต้องพูดถึง ปล่อยน้ำแข็งละลายทุกที5555
*65 ของปั่น ตระกูลเบอรี่ เปรี้ยวและหวานมากกก รอน้ำแข็งละลายก็ไม่หายเปรี้ยววว
*77 พนักงานไม่ควรยื้อเวลาในการทำรายการนานๆเพื่อให้ลูกค้านั่งรอนานจนหิวน้ำนะคะ
*78 โกโก้เย็น ตอนชง ถ้ามึงจะละลายน้ำแล้วเป็นตะกอนนอนเล่นก้นแก้วกูว่ามึงเอาไปต้มล่วงหน้าแล้วคนให้มันละลายกับน้ำ ใส่หม้อไว้รินหรือรอตักใส่แก้วเหอะ ดูดที ผงโกโก้ตะกอนจะติดคอตาย สำลักแล้วสำลักอีก
*116 น้ำแข็งเค็ม
*155 ชาไทยที่เป็นน้ำแข็งป่นน้ำน้อยไป ดูดแปปเดียวหมด
*172 น้ำแข็งลดๆลงบ้างนะคะทั้งเมนูแบบปั่นและแบบเย็น
*195 อยากลดน้ำแข็งชาไทยหน่อยค่ะ มันแน่นเกิ๊น
*220 น้ำแข็งที่ใส่ นึกว่าสั่งชาเย็นพิเศษน้ำแข็ง เยอะกว่าน้ำที้างอีก
*224 รสชาติ น้ำแข็งจะอัดเยอะไปไหน
*343 รสชาติห่วยทุกน้ำเลย ใช้แต้มทรูแลกเพิ่มตังยังผิดหวัง


In [29]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(index=6, options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), value=6)

*10 ['ไทย', 'น้ำ', 'แข็ง', 'แน่นแก้ว', 'ขนาด', 'กิน', 'แป๊บ', 'แก้ว', 'น้ำ', 'แข็ง', 'แก้ว']
*32 ['แทบ', 'รสชาติ', 'ราคา', 'ซื้อ', 'กิน', 'แก้ว', 'เอิ่ม', 'แย่']
*35 ['สิทธิ์', 'ทรู', 'แถม', 'ชาไทย', 'ไหม', 'จ่าย', 'แก้ว', 'แพง', 'แก้ว', 'สอง', 'ชา', 'ไทย', 'ไม']
*85 ['รสชาติ', 'แย่', 'จืด', 'แก้ว', 'โอเค']
*95 ['พนักงาน', 'หนา', 'ผม', 'หน้า', 'แก้ว', 'นม']
*171 ['แก้ว']
*176 ['แก้ว', 'สวย', 'รู้สึก', 'แปลก', 'ดี', 'สี', 'เครื่อง', 'ดื่ม', 'ร้าน']
*241 ['เหมือน', 'น้ำ', 'ล้าง', 'แก้ว', 'แถม']


In [ ]:
seed = 0
compare = 0

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))

print(idx_corpus[seed])
print(corpus[seed])
print(idx_corpus[compare])
print(corpus[compare])